In [35]:
import pandas as pd
import numpy as np
import os
import glob
import re

# Getting Weather Data

The rain data was stored in the NOAA data as a column called AU1. The temperature data was stored in a column called TMP. I pulled those two columns, and the date clumn for each years file, then combined all the data into as single dataframe

In [36]:
all_weather = []
for f in glob.glob('*.csv'):
        data = pd.read_csv(f)
        data = data.loc[:,['DATE','TMP','AU1']]
        all_weather.append(data)
        
all_weather = pd.concat(all_weather)

/Users/charlesphillips/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (42,43,44,45,54,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/charlesphillips/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (44,45,54,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/charlesphillips/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (36,42,43,44,45,50,91) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/charlesphillips/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (42,43,86) have mixed 

## Getting the rain info out of AU1
See weather data info file for the formatting of AU1. I pull the remevant infgormation ourt of it with the get_rain function. Retures 1 if it was raining during that measurement, 0 if not

In [37]:
def get_rain(r):
    if  not pd.isnull(r['AU1']):
        s = r['AU1']
        s =s.split(',')
        if s[2] in ['01','02']:
            return 1
    return 0
    


In [38]:
all_weather['rain'] = all_weather.apply(get_rain, axis = 1);


# Simplifying the data
I will round the hour to the nearest minute, so that we will have measurements on the hour for every hour of every day of every year.

Once i round, i then group the data by Date. Since i rounded the time, there will be Date groups with multiple values, so to choose the 'representative' rain value for that hour, i take the max rain value and the min temperature value (I will explain why min in next section)


In [27]:
all_weather['DATE'] = all_weather['DATE'].apply(lambda x : pd.to_datetime(x).floor('h')).astype(str)
all_weather = all_weather.groupby('DATE')['rain','TMP'].agg({'rain': np.max, 'TMP' : np.min})
all_weather.reset_index(inplace = True)
all_weather.head()

<ipython-input-27-251cef98433a>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  all_weather = all_weather.groupby('DATE')['rain','TMP'].agg({'rain': np.max, 'TMP' : np.min})


,DATE,rain,TMP
0,2013-01-01 00:00:00,0,"+0028,5"
1,2013-01-01 01:00:00,0,"+0028,5"
2,2013-01-01 02:00:00,0,"+0033,5"
3,2013-01-01 03:00:00,0,"+0033,5"
4,2013-01-01 04:00:00,0,"+0033,5"


# Removing characters from Temperature Reading
The temperature readings have letter values next to them (see weather documentation). We will all characters that do not pertain to the temperature reading. We also divide by 10 since the data dictionary says that every datapoint was multiplied by a factor of 10

In [28]:
all_weather['TMP'] = all_weather['TMP'].apply(lambda x: x.split(',')[0])
all_weather['TMP'] = all_weather['TMP'].apply(lambda x: float(re.sub('[A-Z+,]','',x))/10)

# Handling missing values (999.99)
A missing value for temperature in this dataset is 999.99. To impute these values, i simply fill the missing temperature with the temperature from the hour before.

In [30]:
all_weather = all_weather.apply(lambda x : np.nan if x['TMP'] > 999 else x,axis = 1)
all_weather.fillna(method = 'bfill',inplace = True)

# Save to parquet for efficient storage

In [33]:
all_weather.to_parquet('all_weather_final.parquet')